In [4]:
import os
import math
import sagemaker
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/util/clustering


# ATRW Clustering

In [5]:
estimator = TensorFlow(entry_point='clustering.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'output-s3': 's3://calvinandpogs-ee148/atrw/detection/annotations/clusters/vgg16places/pca-kmeans'
                        })

In [ ]:
estimator.fit({'images': f's3://{bucket}/atrw/detection/train/'})

# LilaBC Clustering

## Full dataset

In [2]:
total_clusters = 61

estimator = TensorFlow(entry_point='clustering.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'output-s3': 's3://calvinandpogs-ee148/lilabc/clusters/full/',
                            'num-clusters': total_clusters
                        })

In [ ]:
estimator.fit({'images': f's3://{bucket}/lilabc/images/labels-full/'})

## Fractional subsets

In [ ]:
%%capture --no-stderr

total_clusters = 60        # To match the script-mistake
sets = [16, 8, 4, 2]
# sets = [16]

for set in sets:
    estimator = TensorFlow(entry_point='clustering.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'output-s3': f's3://calvinandpogs-ee148/lilabc/clusters/frac{set}/',
                                'num-clusters': math.ceil(float(total_clusters) / set)
                            })
    estimator.fit({'images': f's3://{bucket}/lilabc/images/labels-{set}/'})